# Why am i getting a periodic system for vacuum?

## See if its a system generator problem

In [1]:
from perses.utils.openeye import *
from perses.annihilation.relative import HybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import copy
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openforcefield.topology import Molecule
from openmmforcefields.generators import SystemGenerator

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
RDKit WARNING: [20:22:03] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

In [3]:
# First thing to do is load the apo protein to mutate...
protein_pdbfile = open("../input/ala_vacuum.pdb", 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms

In [4]:
# Load the ligand, if present
molecules = []
ligand_filename = "../input/ser_vacuum.pdb"
if ligand_filename:
    if ligand_filename.endswith('.sdf'): # small molecule
        ligand_mol = createOEMolFromSDF(ligand_filename, index=ligand_index)
        ligand_mol = generate_unique_atom_names(ligand_mol)
        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
        ligand_n_atoms = ligand_md_topology.n_atoms

    elif ligand_filename.endswith('pdb'): # protein
        ligand_pdbfile = open(ligand_filename, 'r')
        ligand_pdb = app.PDBFile(ligand_pdbfile)
        ligand_pdbfile.close()
        ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
            ligand_pdb.topology)
        ligand_n_atoms = ligand_md_topology.n_atoms
    
    # Now create a complex
    complex_md_topology = protein_md_topology.join(ligand_md_topology)
    complex_topology = complex_md_topology.to_openmm()
    complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
    complex_positions[:protein_n_atoms, :] = protein_positions
    complex_positions[protein_n_atoms:, :] = ligand_positions

In [14]:
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
# nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff}
nonperiodic_forcefield_kwargs = {}
system_generator = SystemGenerator(forcefields = forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                cache=None)

DEBUG:perses.forcefields.system_generators:Trying GAFFTemplateGenerator to load openff-1.0.0
DEBUG:perses.forcefields.system_generators:  GAFFTemplateGenerator cannot load openff-1.0.0
DEBUG:perses.forcefields.system_generators:Specified 'forcefield' (openff-1.0.0) must be one of ['gaff-1.4', 'gaff-1.8', 'gaff-1.81', 'gaff-2.1', 'gaff-2.11']
DEBUG:perses.forcefields.system_generators:Trying SMIRNOFFTemplateGenerator to load openff-1.0.0


In [20]:
system_generator.periodic_forcefield_kwargs

{'nonbondedMethod': PME}

In [21]:
system_generator.nonperiodic_forcefield_kwargs

{'nonbondedMethod': NoCutoff}

In [16]:
system = system_generator.create_system(complex_topology)

In [17]:
system.usesPeriodicBoundaryConditions()

False

In [ ]:
sysetm.

## Reproduce the problem with PointMutationExecutor

In [1]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
import simtk.openmm.app as app
apo_delivery = PointMutationExecutor(f"../input/ala_vacuum.pdb", 
                              '1', 
                              '2', 
                              'SER'
                             )

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


16777992


RDKit WARNING: [22:44:47] Enabling RDKit 2019.09.3 jupyter extensions
INFO:root:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using core to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{0: 0, 2: 1, 3: 2}, {0: 0, 2: 1, 3: 3}, {0: 0, 3: 2, 2: 3}]
INFO:proposal_generator:{0: 0, 2: 1, 3: 2}
INFO:proposal_generator:{0: 0, 2: 1, 3: 3}
INFO:proposal_generator:{0: 0, 3: 2, 2: 3}
INFO:proposal_generator:Finding best map using matching_criterion index
INFO:proposal_generator:3 have 3 core atoms. Using matching_criterion index to return the best of those
INFO:proposal_generator:{0: 0, 2: 1, 3: 3}
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be pl

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

INFO:geometry:total reduced energy added from growth system: 22.707579865893052
INFO:geometry:final reduced energy 31.825508050891976
INFO:geometry:sum of energies: 31.825507612012046
INFO:geometry:magnitude of difference in the energies: 4.3887992973168366e-07
INFO:geometry:Final logp_proposal: 25.72767226898851


added energy components: [('CustomBondForce', 0.0006997761422412573), ('CustomAngleForce', 1.9747549513848426), ('CustomTorsionForce', 7.25085640852447), ('CustomBondForce', 13.481268729841489)]


INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.5648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.5648, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.5648), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions 

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
			HarmonicBondForce: 0.03336689198711389
			HarmonicAngleForce: 2.1347394429482724
			PeriodicTorsionForce: 16.176131477654224
			NonbondedForce: -40.48578205587068
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 0.032667115844874646
			HarmonicBondForce: 0.6181061041895441
			CustomAngleForce: 0.15998449156336056
			HarmonicAngleForce: 8.016839247805873
			CustomTorsionForce: 0.0
			PeriodicTorsionForce: 26.87994502099509
			NonbondedForce: -64.87357466750167
			CustomNonbondedForce: -8.678095883557813
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 0.032667115844874646
			HarmonicBondForce: 0.6181061041895441
			CustomAngleForce: 0.15998449156336056
			HarmonicAngleFor

			HarmonicBondForce: 0.6500734438921796
			HarmonicAngleForce: 6.202068787984321
			PeriodicTorsionForce: 19.62908968205162
			NonbondedForce: -42.13294377342093
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0


In [17]:
apo_delivery.apo_htf.hybrid_system.usesPeriodicBoundaryConditions()

False

# Debug through PointMutationExecutor 

In [22]:
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [2]:
from perses.utils.openeye import *
from perses.annihilation.relative import HybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import copy
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openforcefield.topology import Molecule
from openmmforcefields.generators import SystemGenerator

In [3]:
ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

In [4]:
# First thing to do is load the apo protein to mutate...
protein_pdbfile = open("../input/mmc2_barstar.pdb", 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms

In [5]:
# Load the ligand, if present
molecules = []
ligand_filename = "../input/mmc2_barnase.pdb"
if ligand_filename:
    if ligand_filename.endswith('.sdf'): # small molecule
        ligand_mol = createOEMolFromSDF(ligand_filename, index=ligand_index)
        ligand_mol = generate_unique_atom_names(ligand_mol)
        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
        ligand_n_atoms = ligand_md_topology.n_atoms

    elif ligand_filename.endswith('pdb'): # protein
        ligand_pdbfile = open(ligand_filename, 'r')
        ligand_pdb = app.PDBFile(ligand_pdbfile)
        ligand_pdbfile.close()
        ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
            ligand_pdb.topology)
        ligand_n_atoms = ligand_md_topology.n_atoms
    
    # Now create a complex
    complex_md_topology = protein_md_topology.join(ligand_md_topology)
    complex_topology = complex_md_topology.to_openmm()
    complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
    complex_positions[:protein_n_atoms, :] = protein_positions
    complex_positions[protein_n_atoms:, :] = ligand_positions

In [6]:
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
nonperiodic_forcefield_kwargs = None
system_generator = SystemGenerator(forcefields = forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                   nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                molecules=molecules,
                                                cache=None)

In [45]:
def _solvate(
           topology,
           positions,
           water_model,
           phase,
           ionic_strength,
           box_dimensions = None):
    """
    Generate a solvated topology, positions, and system for a given input topology and positions.
    For generating the system, the forcefield files provided in the constructor will be used.

    Parameters
    ----------
    topology : app.Topology
        Topology of the system to solvate
    positions : [n, 3] ndarray of Quantity nm
        the positions of the unsolvated system
    forcefield : SystemGenerator.forcefield
        forcefield file of solvent to add
    water_model : str
        solvent model to use for solvation
    phase : str
        if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
    ionic_strength : float * unit.molar
        the total concentration of ions (both positive and negative) to add using Modeller.
        This does not include ions that are added to neutralize the system.
        Note that only monovalent ions are currently supported.

    Returns
    -------
    solvated_topology : app.Topology
        Topology of the system with added waters
    solvated_positions : [n + 3(n_waters), 3] ndarray of Quantity nm
        Solvated positions
    solvated_system : openmm.System
        The parameterized system, containing a barostat if one was specified.
    """
    modeller = app.Modeller(topology, positions)

    # Now we have to add missing atoms
    if phase != 'vacuum':
        _logger.info(f"solvating at {ionic_strength} using {water_model}")
        if not box_dimensions:
            modeller.addSolvent(system_generator.forcefield, model=water_model, padding=0.9 * unit.nanometers, ionicStrength=ionic_strength)
            print("getting here 1")
        else:
            modeller.addSolvent(system_generator.forcefield, model=water_model, boxSize=box_dimensions, ionicStrength=ionic_strength)
            print("getting here 2")
    else:
        pass

    solvated_topology = modeller.getTopology()
#     solvated_topology.setUnitCellDimensions(box_dimensions)
    solvated_positions = modeller.getPositions()

    # Canonicalize the solvated positions: turn tuples into np.array
    solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
    solvated_system = system_generator.create_system(solvated_topology)

    return solvated_topology, solvated_positions, solvated_system


In [31]:
def _solvate(topology,
                    positions,
                    model,
                    phase):
    """
    Generate a solvated topology, positions, and system for a given input topology and positions.
    For generating the system, the forcefield files provided in the constructor will be used.

    Parameters
    ----------
    topology : app.Topology
        Topology of the system to solvate
    positions : [n, 3] ndarray of Quantity nm
        the positions of the unsolvated system
    forcefield : SystemGenerator.forcefield
        forcefield file of solvent to add
    model : str, default 'tip3p'
        solvent model to use for solvation

    Returns
    -------
    solvated_topology : app.Topology
        Topology of the system with added waters
    solvated_positions : [n + 3(n_waters), 3] ndarray of Quantity nm
        Solvated positions
    solvated_system : openmm.System
        The parameterized system, containing a barostat if one was specified.
    """
    modeller = app.Modeller(topology, positions)


    # Now we have to add missing atoms
    if phase != 'vacuum':
        modeller.addSolvent(system_generator.forcefield, model=model, padding=1.0 * unit.nanometers, ionicStrength=0.05*unit.molar)
    else:
        pass

    solvated_topology = modeller.getTopology()
    solvated_positions = modeller.getPositions()

    # Canonicalize the solvated positions: turn tuples into np.array
    solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
    solvated_system = system_generator.create_system(solvated_topology)

    return solvated_topology, solvated_positions, solvated_system


In [32]:
apo_input = list(_solvate(protein_topology, protein_positions, 'tip3p', phase='complex'))
inputs = [apo_input]
if ligand_filename:
    inputs.append(_solvate(complex_topology, complex_positions, 'tip3p', phase='complex'))

In [34]:
inputs[1][2].usesPeriodicBoundaryConditions()

True

In [51]:
protein_topology.getUnitCellDimensions()

In [46]:
ionic_strength = 0.15 * unit.molar
complex_box_dimensions = None
apo_box_dimensions = None
water_model = 'tip3p'
phase = 'complex'
apo_input = list(_solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
inputs = [apo_input]
if ligand_filename:
    inputs.append(_solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))


INFO:root:solvating at 0.15 M using tip3p


getting here 1


INFO:root:solvating at 0.15 M using tip3p


getting here 1


In [47]:
inputs[0][2].usesPeriodicBoundaryConditions()

True

In [9]:
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)

In [15]:
conduct_endstate_validation = True
htfs = []
for i, (top, pos, sys) in enumerate(inputs):
    print(i)
    point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                         system_generator=system_generator,
                                                         chain_id='1', # Denote the chain id allowed to mutate (it's always a string variable)
                                                         max_point_mutants=1,
                                                         residues_allowed_to_mutate=['29'], # The residue ids allowed to mutate
                                                         allowed_mutations=[('29', 'ALA')], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                         aggregate=True) # Always allow aggregation

    topology_proposal = point_mutation_engine.propose(sys, top)

    new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta, validate_energy_bookkeeping = False)
    logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta, validate_energy_bookkeeping = False)

    forward_htf = HybridTopologyFactory(topology_proposal=topology_proposal,
                                         current_positions=pos,
                                         new_positions=new_positions,
                                         use_dispersion_correction=False,
                                         functions=None,
                                         softcore_alpha=None,
                                         bond_softening_constant=1.0,
                                         angle_softening_constant=1.0,
                                         soften_only_new=False,
                                         neglected_new_angle_terms=[],
                                         neglected_old_angle_terms=[],
                                         softcore_LJ_v2=True,
                                         softcore_electrostatics=True,
                                         softcore_LJ_v2_alpha=0.85,
                                         softcore_electrostatics_alpha=0.3,
                                         softcore_sigma_Q=1.0,
                                         interpolate_old_and_new_14s=False,
                                         omitted_terms=None)
    if not topology_proposal.unique_new_atoms:
        assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
        assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
        vacuum_added_valence_energy = 0.0
    else:
        added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

    if not topology_proposal.unique_old_atoms:
        assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
        assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
        subtracted_valence_energy = 0.0
    else:
        subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


    if conduct_endstate_validation:
        zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta=beta, ENERGY_THRESHOLD=ENERGY_THRESHOLD)
        assert zero_state_error < ENERGY_THRESHOLD, f"Reduced potential difference of the nonalchemical and alchemical Lambda = 0 state is above the threshold ({ENERGY_THRESHOLD}): {zero_state_error}"
        assert one_state_error < ENERGY_THRESHOLD, f"Reduced potential difference of the nonalchemical and alchemical Lambda = 1 state is above the threshold ({ENERGY_THRESHOLD}): {one_state_error}"
    else:
        pass


INFO:proposal_generator:	Conducting polymer point mutation proposal...


0


INFO:proposal_generator:Using core to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{0: 0, 1: 8, 2: 9}, {0: 0, 1: 8, 3: 9}, {0: 0, 2: 8, 3: 9}]
INFO:proposal_generator:{0: 0, 1: 8, 2: 9}
INFO:proposal_generator:{0: 0, 1: 8, 3: 9}
INFO:proposal_generator:{0: 0, 2: 8, 3: 9}
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_generator:3 have 3 core atoms. Using matching_criterion name to return the best of those
INFO:proposal_generator:{0: 0, 1: 8, 2: 9}
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 4
INFO:geometry:Atom index proposal order is [470, 475, 474, 473]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 7.525144014625207
INFO:geometry:final reduced energy 4802.949852557921
INFO:geometry:sum of energies: 4805.006563876056
INFO:geometry:magnitude of difference in the energies: 2.0567113181354078
INFO:geometry:Final logp_proposal: 112.85335644128457


added energy components: [('CustomBondForce', 0.8890211788717974), ('CustomAngleForce', 1.3082326281445291), ('CustomTorsionForce', 11.549703246497025), ('CustomBondForce', -6.221813038888139)]


INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.9997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.9997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.9997), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions 

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
			HarmonicBondForce: 189.69655643562666
			HarmonicAngleForce: 791.7773839676565
			PeriodicTorsionForce: 3830.521840699088
			NonbondedForce: -65426.689422382704
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 2.77893418543332
			HarmonicBondForce: 187.2875900023216
			CustomAngleForce: 7.841561373685756
			HarmonicAngleForce: 785.2771391863009
			CustomTorsionForce: 0.05405409160040507
			PeriodicTorsionForce: 3838.66580662559
			NonbondedForce: -65302.4921218216
			CustomNonbondedForce: -109.2602961088798
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 2.77893418543332
			HarmonicBondForce: 187.2875900023216
			CustomAngleForce: 7.841561373685756
			HarmonicAngleForce

In [13]:
forward_htf.hybrid_system.usesPeriodicBoundaryConditions()

True